In [2]:
import psycopg2
import psycopg2.extras
import csv
import pandas as pd
from datetime import datetime, timedelta
import json

In [4]:
dconnector="host=34.66.221.94 port=5432 dbname=ecobici user=postgres password=password"
nameOfTheFile = "ecobici_may"

In [4]:
def data_to_dict(q):
    conn = psycopg2.connect(dconnector)
    cur = conn.cursor(cursor_factory = psycopg2.extras.RealDictCursor)
    cur.execute(q)
    fulldf = cur.fetchall()
    df=[{
        "id":int(i["id"]),
        "genero_usuario":str(i['genero_usuario']),
        "edad_usuario":float(i["edad_usuario"]),
        "bici":str(i['bici']),
        "ciclo_estacion_retiro":int(i['ciclo_estacion_retiro']),
        "ciclo_estacion_arribo":int(i['ciclo_estacionarribo']),
        "full_date_retiro":str(i['ciclo_estacionarribo']),
        "full_date_arribo":str(i['full_date_aribo']),
        "mes":int(i['mes']),
        "hora":int(i["hora"]),
        "time_delta":float(i["time_delta"]),
        "viaje":str(i["viaje"]),
        "name_retiro":str(i["name_retiro"]),
        "location_lat_retiro":float(i["location_lat_retiro"]),
        "location_lon_retiro":float(i["location_lon_retiro"]),
        "name_arribo":str(i["name_arribo"]),
        "location_lat_arribo":float(i["location_lat_arribo"]),
        "location_lon_arribo":float(i["location_lon_arribo"]),
        "location_dist":round(float(i["location_dist"]),2)
    } for i in fulldf]
    dfII = [df[i] for i,x in enumerate(df)]
    return dfII

d = data_to_dict(q=f"select * from {nameOfTheFile} limit 5")

In [5]:
trips_query = """
SELECT 
	full_date_retiro, COUNT(*) as viajes_totales 
FROM 
	public.ecobici_may 
WHERE 
	full_date_retiro >= '2021-07-01'
group by 
	full_date_retiro 
order by 
	full_date_retiro
"""

def total_trips_per_day(q):
    conn = psycopg2.connect(dconnector)
    cur = conn.cursor(cursor_factory = psycopg2.extras.RealDictCursor)
    cur.execute(q)
    fulldf = cur.fetchall()
    listOfDates=[i["full_date_retiro"] for i in fulldf]
    listOfTrips = [int(i["viajes_totales"]) for i in fulldf]
    
    tripsChart= {
        "listOfDates":listOfDates,
        "listOfTrips":listOfTrips
    }

    return tripsChart

c = total_trips_per_day(q=trips_query)

In [17]:
def gender_trips(fileName="ecobici_may"):
    q= f"""
        SELECT 
            genero_usuario, COUNT(*) as pct
        FROM 
            public.{fileName} 
        WHERE 
            full_date_retiro >= '2021-07-01'
        GROUP BY genero_usuario
        """
    conn = psycopg2.connect(dconnector)
    cur = conn.cursor(cursor_factory = psycopg2.extras.RealDictCursor)
    cur.execute(q)
    fulldf = cur.fetchall()
    listOfDates=[str(i["genero_usuario"]) for i in fulldf]
    listOfTrips = [int(i["pct"]) for i in fulldf]

    total = 0
    list1 = listOfTrips
    
    # Iterate each element in list
    # and add them in variable total
    for ele in range(0, len(list1)):
        total = total + list1[ele]
    
    get_pct = [round((i/total)*100) for i in listOfTrips]
    
    genderChart= {
        "genero":listOfDates,
        "tripsGender":get_pct
    }
    
    return genderChart

l = gender_trips()

In [18]:
l

{'genero': ['X', 'M', 'F'], 'tripsGender': [5, 68, 27]}